# 🧠 Simple RNN from Scratch in Python (No Libraries)
This notebook demonstrates how a basic **Recurrent Neural Network (RNN)** works for word prediction using only **core Python**. We use simple sentences as training data and implement each part manually — from forward pass to prediction.

The goal is to predict the **next word** in a sentence based on the current one.

---
**Key Concepts:**
- One-hot encoding for words
- Hidden state memory (recurrent structure)
- `tanh` activation and `softmax` output
- Matrix operations manually coded


In [1]:
# Sample training data: simple sentences
data = [
    "hello how are you",
    "how are you doing",
    "are you doing well"
]

# Build vocabulary and mapping
words = set(" ".join(data).split())
word_to_idx = {word: i for i, word in enumerate(words)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

vocab_size = len(words)


### 🧾 Why Word Indexing?
We need to convert text into numerical format so the RNN can process it.

- We create a **word-to-index mapping** to assign a unique integer to each word in our vocabulary.
- Later, we use **one-hot encoding** to convert each word into a vector.

🧮 Vocabulary size = total unique words in dataset.


In [21]:
import math
import random

# Initialize RNN parameters (weights and biases)
def init_weights(hidden_size, vocab_size):
    Wx = [[random.uniform(-1, 1) for _ in range(hidden_size)] for _ in range(vocab_size)]  # Input to hidden
    Wh = [[random.uniform(-1, 1) for _ in range(hidden_size)] for _ in range(hidden_size)] # Hidden to hidden
    Wy = [[random.uniform(-1, 1) for _ in range(vocab_size)] for _ in range(hidden_size)]  # Hidden to output
    bh = [0] * hidden_size  # Hidden bias
    by = [0] * vocab_size   # Output bias
    return Wx, Wh, Wy, bh, by


### ⚙️ Why These Weights and Biases?

We need trainable parameters for the RNN:

- **Wx**: Input-to-hidden weights (shapes the input word into hidden form)
- **Wh**: Hidden-to-hidden weights (keeps memory of past inputs)
- **Wy**: Hidden-to-output weights (maps hidden state to output prediction)
- **bh, by**: Bias vectors for hidden and output layers

🔢 These are initialized randomly, as in real neural networks.


In [22]:
# Activation function: tanh
def tanh(x):
    return [math.tanh(i) for i in x]

# Softmax for output prediction
def softmax(x):
    e_x = [math.exp(i) for i in x]
    total = sum(e_x)
    return [i / total for i in e_x]

# Matrix-vector multiplication
def matvec_mul(mat, vec):
    return [sum(m * v for m, v in zip(row, vec)) for row in mat]

# Vector addition
def vec_add(v1, v2):
    return [a + b for a, b in zip(v1, v2)]


### 🧮 Why `tanh` and `softmax`?

- **`tanh` activation** is used to control the hidden state: it outputs values between -1 and 1, which helps smooth gradients during learning.
  
  $$
  \text{tanh}(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
  $$

- **`softmax`** converts raw output scores into probabilities:

  $$
  \text{softmax}(z_i) = \frac{e^{z_i}}{\sum_j e^{z_j}}
  $$

🔢 This lets us pick the most likely next word.


In [23]:
# One RNN time step
def rnn_step(x, h_prev, Wx, Wh, Wy, bh, by):
    h_t = tanh(vec_add(matvec_mul(Wx, x), vec_add(matvec_mul(Wh, h_prev), bh)))
    y_t = vec_add(matvec_mul(Wy, h_t), by)
    y_pred = softmax(y_t)
    return h_t, y_pred


### 🔁 What happens in one RNN step?

Given input word vector `x` and previous hidden state `h_prev`:

1. **Hidden state update**:
   \[
   h_t = \tanh(W_x \cdot x + W_h \cdot h_{\text{prev}} + b_h)
   \]

2. **Output calculation**:
   \[
   y = W_y \cdot h_t + b_y
   \]

3. **Prediction (probability)**:
   \[
   \text{softmax}(y)
   \]

This is how memory flows across words!
### 🔁 What happens in one RNN step?

Given input word vector `x` and previous hidden state `h_prev`:

1. **Hidden state update**:
   $$
   h_t = \tanh(W_x \cdot x + W_h \cdot h_{\text{prev}} + b_h)
   $$

2. **Output calculation**:
  $$
   y = W_y \cdot h_t + b_y
  $$

3. **Prediction (probability)**:
  $$
   \text{softmax}(y)
 $$

This is how memory flows across words!


In [27]:
# One-hot encode word
def one_hot(index, size):
    vec = [0] * size
    vec[index] = 1
    return vec

hidden_size = 8
Wx, Wh, Wy, bh, by = init_weights(hidden_size, vocab_size)

# Simple training loop (forward only)
for sentence in data:
    words_list = sentence.split()
    h_prev = [0] * hidden_size

    for i in range(len(words_list) - 1):
        current_word = words_list[i]
        next_word = words_list[i + 1]

        x = one_hot(word_to_idx[current_word], vocab_size)
        h_prev, y_pred = rnn_step(x, h_prev, Wx, Wh, Wy, bh, by)

        predicted_idx = y_pred.index(max(y_pred))
        predicted_word = idx_to_word[predicted_idx]

        print(f"Input: '{current_word}' → Predicted: '{predicted_word}' (Target: '{next_word}')")


Input: 'hello' → Predicted: 'doing' (Target: 'how')
Input: 'how' → Predicted: 'are' (Target: 'are')
Input: 'are' → Predicted: 'how' (Target: 'you')
Input: 'how' → Predicted: 'how' (Target: 'are')
Input: 'are' → Predicted: 'hello' (Target: 'you')
Input: 'you' → Predicted: 'hello' (Target: 'doing')
Input: 'are' → Predicted: 'doing' (Target: 'you')
Input: 'you' → Predicted: 'hello' (Target: 'doing')
Input: 'doing' → Predicted: 'hello' (Target: 'well')


### 🎯 What Does the Output Mean?

Each line shows:

- The **current input word**
- The **predicted next word** from the RNN
- The **actual target** next word from sentence

This shows how well the RNN has learned patterns, even **without backpropagation** (we're just forwarding and predicting here).



## 🔁 Backpropagation in RNN – Explained with Math and Chain Rule

In this section, we walk through the **backward pass (backpropagation)** in a simple RNN — to update weights using the error between predicted and target output.

---

### 📌 Goal of Backpropagation
To compute gradients (∂Loss/∂W) for all weights so we can **reduce the loss** using **gradient descent**.

---

## 🔄 Forward Recap
At each time step:

1. Hidden state update:
$$
h_t = \tanh(W_x \cdot x + W_h \cdot h_{t-1} + b_h)
$$

2. Output prediction:
$$
y = W_y \cdot h_t + b_y
$$

3. Softmax for probabilities:
$$
\hat{y} = \text{softmax}(y)
$$

4. Loss (Cross-Entropy):
$$
L = -\log(\hat{y}_{\text{target}})
$$

---

## 🔙 Backpropagation: Step by Step

We use **chain rule** to propagate gradients backward through each operation.

---

### 🔹 1. Gradient of Loss w.r.t Softmax Output (Logits)

Let:
- \( \hat{y} \) be softmax output
- \( y_{\text{target}} \) be the true one-hot label

The derivative of cross-entropy loss:
$$
\frac{\partial L}{\partial \hat{y}_i} = \hat{y}_i - y_{\text{target},i}
$$

This is done via:
```python
dy = y_pred[:]      # Clone prediction
dy[target_idx] -= 1 # Subtract 1 at true class
````

---

### 🔹 2. Gradient w\.r.t. Output Layer Weights (Wy)

We compute:

$$
\frac{\partial L}{\partial W_y} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial W_y}
\Rightarrow \text{outer product of } dy \text{ and } h_t
$$

Each element:

$$
\frac{\partial L}{\partial W_y[i][j]} = dy[i] \cdot h_t[j]
$$

```python
for i_v in range(vocab_size):
    for j_h in range(hidden_size):
        Wy[i_v][j_h] -= learning_rate * dy[i_v] * h_t[j_h]
```

---

### 🔹 3. Gradient w\.r.t. Output Bias (by)

Since:

$$
y = W_y \cdot h_t + b_y \Rightarrow \frac{\partial L}{\partial b_y[i]} = dy[i]
$$

```python
for i_v in range(vocab_size):
    by[i_v] -= learning_rate * dy[i_v]
```

---

### 🔹 4. Gradient w\.r.t. Hidden State $h_t$

We propagate error from output back to hidden state:

$$
\frac{\partial L}{\partial h_t[j]} = \sum_{i} W_y[i][j] \cdot dy[i]
$$

```python
dh = [sum(Wy[i_v][j_h] * dy[i_v] for i_v in range(vocab_size)) for j_h in range(hidden_size)]
```

---

### 🔹 5. Derivative of tanh Activation

Since:

$$
h_t = \tanh(z), \quad \frac{d}{dz} \tanh(z) = 1 - \tanh^2(z)
$$

$$
\frac{\partial L}{\partial z} = \frac{\partial L}{\partial h_t} \cdot (1 - h_t^2)
$$

```python
dh_raw = [dh[j_h] * (1 - h_t[j_h] ** 2) for j_h in range(hidden_size)]
```

---

### 🔹 6. Gradient w\.r.t. Input Weights (Wx)

$$
\frac{\partial L}{\partial W_x[i][j]} = dh_{\text{raw}}[i] \cdot x[j]
$$

```python
for i_h in range(hidden_size):
    for j_v in range(vocab_size):
        Wx[i_h][j_v] -= learning_rate * dh_raw[i_h] * x[j_v]
```

---

### 🔹 7. Gradient w\.r.t. Hidden Bias (bh)

$$
\frac{\partial L}{\partial b_h[i]} = dh_{\text{raw}}[i]
$$

```python
for i_h in range(hidden_size):
    bh[i_h] -= learning_rate * dh_raw[i_h]
```

---

### 🔹 8. Gradient w\.r.t. Recurrent Weights (Wh)

$$
\frac{\partial L}{\partial W_h[i][j]} = dh_{\text{raw}}[i] \cdot h_{\text{prev}}[j]
$$

```python
for i_h in range(hidden_size):
    for j_h_prev in range(hidden_size):
        Wh[i_h][j_h_prev] -= learning_rate * dh_raw[i_h] * h_prev[j_h_prev]
```

---

## 📉 Why This Matters

Backpropagation updates all weights to minimize the loss between predicted and target word. Without this step, the model won’t **learn or improve**.

Every gradient step uses the **chain rule** to flow error backward:

$$
\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial x}
$$

---

✅ With each epoch, we reduce the total loss and improve predictions!






In [19]:
import random
import math

# ========== 1. Prepare data ==========
data = [
    "hello how are you",
    "how are you doing",
    "are you doing well"
]

# Word to index mapping
words = sorted(set(" ".join(data).split()))
word_to_idx = {w: i for i, w in enumerate(words)}
idx_to_word = {i: w for w, i in word_to_idx.items()}
vocab_size = len(words)
print(f"Vocabulary Size: {vocab_size}") # Debug print
print(f"Words: {words}") # Debug print

# One-hot encoding
def one_hot(index, size):
    vec = [0] * size
    vec[index] = 1
    return vec

# ========== 2. Helper functions ==========
def matvec_mul(matrix, vector):
    # Multiply matrix (MxN) by vector (N,) -> result (M,)
    # result[i] = sum_j matrix[i][j] * vector[j]
    return [sum(m * v for m, v in zip(row, vector)) for row in matrix]

def vec_add(v1, v2):
    return [a + b for a, b in zip(v1, v2)]

def tanh(vec):
    return [math.tanh(v) for v in vec]

# Note: dtanh calculation integrated below for efficiency

def softmax(x):
    # Numerical stability fix
    max_x = max(x)
    e_x = [math.exp(i - max_x) for i in x] # Subtract max for stability
    total = sum(e_x)
    if total == 0:
        return [1.0 / len(x)] * len(x) # Avoid division by zero
    return [i / total for i in e_x]

def cross_entropy(predicted, target_idx):
    # Add small epsilon to prevent log(0)
    return -math.log(predicted[target_idx] + 1e-9)

# ========== 3. Initialize parameters ==========
hidden_size = 8
learning_rate = 0.1

# Weight matrix dimensions: [Output_Size][Input_Size] for matvec_mul(W, v)
# Wx: Input to Hidden (hidden_size x vocab_size)
Wx = [[random.uniform(-0.1, 0.1) for _ in range(vocab_size)] for _ in range(hidden_size)]
# Wh: Hidden to Hidden (hidden_size x hidden_size)
Wh = [[random.uniform(-0.1, 0.1) for _ in range(hidden_size)] for _ in range(hidden_size)]
# Wy: Hidden to Output (vocab_size x hidden_size)
Wy = [[random.uniform(-0.1, 0.1) for _ in range(hidden_size)] for _ in range(vocab_size)]

bh = [0] * hidden_size
by = [0] * vocab_size

# Debug prints for dimensions
print(f"Wx dimensions: {len(Wx)} x {len(Wx[0]) if Wx else 0} (expected {hidden_size} x {vocab_size})")
print(f"Wh dimensions: {len(Wh)} x {len(Wh[0]) if Wh else 0} (expected {hidden_size} x {hidden_size})")
print(f"Wy dimensions: {len(Wy)} x {len(Wy[0]) if Wy else 0} (expected {vocab_size} x {hidden_size})")
print(f"bh dimensions: {len(bh)} (expected {hidden_size})")
print(f"by dimensions: {len(by)} (expected {vocab_size})")


# ========== 4. Training ==========
for epoch in range(100):
    total_loss = 0
    for sentence in data:
        words_list = sentence.split()
        h_prev = [0] * hidden_size

        for i in range(len(words_list) - 1):
            x_word = words_list[i]
            target_word = words_list[i + 1]

            x = one_hot(word_to_idx[x_word], vocab_size)
            target_idx = word_to_idx[target_word]

            # ------ Forward ------
            h_in = vec_add(matvec_mul(Wx, x), vec_add(matvec_mul(Wh, h_prev), bh))
            h_t = tanh(h_in)
            y_raw = vec_add(matvec_mul(Wy, h_t), by)
            y_pred = softmax(y_raw)

            loss = cross_entropy(y_pred, target_idx)
            total_loss += loss

            # ------ Backward ------
            dy = y_pred[:]
            # Check size consistency
            if len(dy) != vocab_size:
                raise ValueError(f"Softmax output size mismatch at epoch {epoch}! Expected {vocab_size}, got {len(dy)}")

            dy[target_idx] -= 1  # gradient of cross-entropy w.r.t. logits (y_raw)

            # Gradients w.r.t. Wy and by
            # dWy = outer(dy, h_t) -> dWy[i][j] = dy[i] * h_t[j]
            # Wy shape: [vocab_size][hidden_size]
            # dy shape: [vocab_size]
            # h_t shape: [hidden_size]
            for i_v in range(vocab_size): # Iterate over vocab/output dimension
                for j_h in range(hidden_size): # Iterate over hidden/input dimension
                    Wy[i_v][j_h] -= learning_rate * dy[i_v] * h_t[j_h]

            # dby = dy
            # by shape: [vocab_size]
            for i_v in range(vocab_size):
                by[i_v] -= learning_rate * dy[i_v]

            # Gradient w.r.t. hidden state (h_t)
            # dh = Wy^T @ dy
            # Wy^T shape: [hidden_size][vocab_size]
            # dy shape: [vocab_size]
            # dh shape: [hidden_size]
            dh = [sum(Wy[i_v][j_h] * dy[i_v] for i_v in range(vocab_size)) for j_h in range(hidden_size)]

            # Gradient before activation (tanh)
            # dh_raw = dh * dtanh(h_in)
            # Using the identity: dtanh(x) = 1 - tanh(x)^2
            # Since h_t = tanh(h_in), we can use h_t directly
            # dh_raw, dh, h_in, h_t shape: [hidden_size]
            dh_raw = [dh[j_h] * (1 - h_t[j_h] ** 2) for j_h in range(hidden_size)]

            # Gradients w.r.t. Wx and bh
            # dWx = outer(dh_raw, x)
            # Wx shape: [hidden_size][vocab_size]
            # dh_raw shape: [hidden_size]
            # x shape: [vocab_size]
            for i_h in range(hidden_size): # Iterate over hidden/output dimension
                for j_v in range(vocab_size): # Iterate over vocab/input dimension
                    Wx[i_h][j_v] -= learning_rate * dh_raw[i_h] * x[j_v]

            # dbh = dh_raw
            # bh shape: [hidden_size]
            for i_h in range(hidden_size):
                bh[i_h] -= learning_rate * dh_raw[i_h]

            # Gradient w.r.t. Wh
            # dWh = outer(dh_raw, h_prev)
            # Wh shape: [hidden_size][hidden_size]
            # dh_raw shape: [hidden_size] (for rows)
            # h_prev shape: [hidden_size] (for columns)
            for i_h in range(hidden_size): # Iterate over current hidden dimension
                for j_h_prev in range(hidden_size): # Iterate over previous hidden dimension
                    Wh[i_h][j_h_prev] -= learning_rate * dh_raw[i_h] * h_prev[j_h_prev]


            h_prev = h_t # Update hidden state for next time step

    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {total_loss:.4f}")

# ========== 5. After Training ==========
print("\n--- Predictions after training ---\n")
for sentence in data:
    words_list = sentence.split()
    h_prev = [0] * hidden_size
    for i in range(len(words_list) - 1):
        current_word = words_list[i]
        next_word = words_list[i + 1]

        x = one_hot(word_to_idx[current_word], vocab_size)
        h_in = vec_add(matvec_mul(Wx, x), vec_add(matvec_mul(Wh, h_prev), bh))
        h_t = tanh(h_in)
        y_raw = vec_add(matvec_mul(Wy, h_t), by)
        y_pred = softmax(y_raw)

        predicted_idx = y_pred.index(max(y_pred))
        predicted_word = idx_to_word[predicted_idx]

        print(f"Input: '{current_word}' → Predicted: '{predicted_word}' (Target: '{next_word}')")

        h_prev = h_t


Vocabulary Size: 6
Words: ['are', 'doing', 'hello', 'how', 'well', 'you']
Wx dimensions: 8 x 6 (expected 8 x 6)
Wh dimensions: 8 x 8 (expected 8 x 8)
Wy dimensions: 6 x 8 (expected 6 x 8)
bh dimensions: 8 (expected 8)
by dimensions: 6 (expected 6)
Epoch 0 | Loss: 16.2611
Epoch 10 | Loss: 11.3341
Epoch 20 | Loss: 3.7659
Epoch 30 | Loss: 1.3022
Epoch 40 | Loss: 0.6056
Epoch 50 | Loss: 0.3696
Epoch 60 | Loss: 0.2599
Epoch 70 | Loss: 0.1982
Epoch 80 | Loss: 0.1591
Epoch 90 | Loss: 0.1323

--- Predictions after training ---

Input: 'hello' → Predicted: 'how' (Target: 'how')
Input: 'how' → Predicted: 'are' (Target: 'are')
Input: 'are' → Predicted: 'you' (Target: 'you')
Input: 'how' → Predicted: 'are' (Target: 'are')
Input: 'are' → Predicted: 'you' (Target: 'you')
Input: 'you' → Predicted: 'doing' (Target: 'doing')
Input: 'are' → Predicted: 'you' (Target: 'you')
Input: 'you' → Predicted: 'doing' (Target: 'doing')
Input: 'doing' → Predicted: 'well' (Target: 'well')
